<a href="https://colab.research.google.com/github/RaIvPa/Improving-BERT-for-Biomedical-QA/blob/main/practice/BERT_base_sentence_classification_with_BioASQ_preprocessing_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install transformers
!pip install datasets transformers[sentencepiece]
!pip install rouge

import transformers
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from datasets import load_dataset
import numpy as np
import csv
from csv import reader
import json
from rouge import Rouge

In [15]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
def create_training_dataset(csv_name, data_file_name, data_num):  
  csv_file = open(csv_name, 'w')
  data_file = open(data_file_name, 'r')

  data_load = json.load(data_file)
  csv_write = csv.writer(csv_file)
  csv_write.writerow(["question", "snippet", "rouge-2", "label"])

  rouge = Rouge()
    
  rows = []
  top_n = 5
  question_num = 0
  for question_set in data_load["questions"]:
    if question_num <= data_num:
      sub_rows = []
      question = question_set["body"]
      top = []
      top_id = {}
      snip_num = 0
      chosen = 0
      for snippet in question_set["snippets"]:
          snip_text = snippet["text"]
          rouge_score = rouge.get_scores(question, snip_text)
          rouge_2_f = rouge_score[0]["rouge-2"]["f"]
          top.append(rouge_2_f)
          # top.sort(reverse=True)
          # if check <= 16:
          #   print(top)
          #   check += 1
          #   # print(top_id)
          #   if len(top) == 0:
          #     top.append(rouge_2_f)
          #     top_id[rouge_2_f] = snip_num
          #   elif len(top) < top_n:
          #     top.append(rouge_2_f)
          #     top_id[rouge_2_f] = snip_num
          #   elif len(top) == top_n:
          #     if rouge_2_f >= top[top_n-1]:
          #       top_id.pop(top[top_n-1])
          #       top.pop(top_n-1)
          #       top.append(rouge_2_f)
          #       top_id[rouge_2_f] = snip_num
          sub_rows.append([question, snip_text, rouge_2_f])
          #   snip_num += 1
          #   print(top_id)
        
      top_indx = sorted(range(len(top)), key=lambda i: top[i], reverse=True)[:5]
      # print(top_indx)

      for indx in top_indx:
        sub_rows[indx].append(1)

      for row in sub_rows:
        if len(row) < 4:
          row.append(0)
        # score = row[2]
        # if len(top) < top_n:
        #   row.append(1)
        # elif score >= top[top_n-1] and len(top) == top_n and chosen < top_n:
        #   row.append(1)
        #   chosen += 1
        # else:
        #   row.append(0)


      rows.append(sub_rows)
    question_num += 1

  for final_row in rows:
    csv_write.writerows(final_row)

  csv_file.close()
  data_file.close()

In [17]:
def create_validation_dataset(csv_name, data_file_name, data_num):  
  csv_file = open(csv_name, 'w')
  data_file = open(data_file_name, 'r')

  data_load = json.load(data_file)
  csv_write = csv.writer(csv_file)
  csv_write.writerow(["question", "snippet", "rouge-2", "label"])

  rouge = Rouge()
    
  rows = []
  top_n = 5
  question_num = 0
  for question_set in data_load["questions"]:
    if question_num > data_num:
      sub_rows = []
      question = question_set["body"]
      top = []
      top_id = {}
      snip_num = 0
      chosen = 0
      for snippet in question_set["snippets"]:
          snip_text = snippet["text"]
          rouge_score = rouge.get_scores(question, snip_text)
          rouge_2_f = rouge_score[0]["rouge-2"]["f"]
          top.append(rouge_2_f)
          # top.sort(reverse=True)
          # if check <= 16:
          #   print(top)
          #   check += 1
          #   # print(top_id)
          #   if len(top) == 0:
          #     top.append(rouge_2_f)
          #     top_id[rouge_2_f] = snip_num
          #   elif len(top) < top_n:
          #     top.append(rouge_2_f)
          #     top_id[rouge_2_f] = snip_num
          #   elif len(top) == top_n:
          #     if rouge_2_f >= top[top_n-1]:
          #       top_id.pop(top[top_n-1])
          #       top.pop(top_n-1)
          #       top.append(rouge_2_f)
          #       top_id[rouge_2_f] = snip_num
          sub_rows.append([question, snip_text, rouge_2_f])
          #   snip_num += 1
          #   print(top_id)
        
      top_indx = sorted(range(len(top)), key=lambda i: top[i], reverse=True)[:5]
      # print(top_indx)

      for indx in top_indx:
        sub_rows[indx].append(1)

      for row in sub_rows:
        if len(row) < 4:
          row.append(0)
        # score = row[2]
        # if len(top) < top_n:
        #   row.append(1)
        # elif score >= top[top_n-1] and len(top) == top_n and chosen < top_n:
        #   row.append(1)
        #   chosen += 1
        # else:
        #   row.append(0)


      rows.append(sub_rows)
    question_num += 1

  for final_row in rows:
    csv_write.writerows(final_row)

  csv_file.close()
  data_file.close()

CSV column format is


*   Question
*   Snippet
*   ROUGE score
*   Label




In [ ]:
dataset_split_num = 250
create_training_dataset('training_data.csv', '/content/drive/MyDrive/Thesis/training9b.json', dataset_split_num)
create_validation_dataset('validation_data.csv', '/content/drive/MyDrive/Thesis/training9b.json', dataset_split_num)

In [ ]:
datasets = load_dataset('csv', data_files={"train": 'training_data.csv', "validation": "validation_data.csv"})
print(datasets)

In [5]:
def tokenize_dataset(dataset):
  encoded = tokenizer(
      dataset["question"],
      dataset["snippet"],
      padding=True,
      truncation=True,
      return_tensors="np",
  )
  return encoded.data

In [6]:
# tok_data = {
#     split: tokenize_dataset(datasets[split]) for split in datasets.keys()
# }

tok_data = tokenize_dataset(datasets["train"])

In [7]:
# print(tok_data["train"])
# print(datasets["train"].features)
# print()
# print(tok_data["validation"])
# print(datasets["validation"].features)

print(tok_data)

{'input_ids': array([[  101,  2003, 28127, ...,     0,     0,     0],
       [  101,  2003, 28127, ...,     0,     0,     0],
       [  101,  2003, 28127, ...,     0,     0,     0],
       ...,
       [  101,  2515,  5255, ...,     0,     0,     0],
       [  101,  2029, 16012, ...,     0,     0,     0],
       [  101,  2029, 16012, ...,     0,     0,     0]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}


In [8]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

In [9]:
bat_size = 8
epoch_num = 8

train_steps = (len(tok_data["input_ids"]) // bat_size) * epoch_num
lr_scheduler = PolynomialDecay(
    initial_learning_rate=1e-4,
    end_learning_rate=0.,
    decay_steps=train_steps
    )

In [10]:
from tensorflow.keras.optimizers import Adam

In [11]:
opt = Adam(learning_rate=lr_scheduler)

In [12]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import Input

Implementing linear regression into the system

In [13]:
new_model_inputs = Input(shape = (322,), dtype='int64')
new_model = (model)(new_model_inputs)
flat_model = Flatten()(new_model[0])
# classifier = Dense(units=len(tok_data["input_ids"]), activation="linear")(flat_model) 
fin_model = Model(inputs=new_model_inputs, outputs=classifier)
fin_model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

KeyboardInterrupt: ignored

In [ ]:
fin_model.compile(optimizer=opt, loss=SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

fin_model.fit(
    tok_data["input_ids"],
    np.array(datasets["train"]["label"]),
    batch_size=bat_size,
)

In [14]:
model.compile(optimizer=opt, loss=SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

model.fit(
    tok_data["input_ids"],
    np.array(datasets["train"]["label"]),
    batch_size=bat_size,
)

479/479 [==============================] - 573s 1s/step - loss: 0.6272 - accuracy: 0.6929


Linear Regression - BERT base model

479/479

loss: 4.4516

accuracy: 0.5377

Logistic Regression - BERT base model (accuracy starts to go down past 370)

479/479

loss: 0.6272

accuracy: 0.6929

In [15]:
predictions = model.predict(tok_data)["logits"]

In [16]:
predictions

TFSequenceClassifierOutput([('logits', array([[-0.26870486, -0.77602845],
                                    [-0.26882064, -0.7761613 ],
                                    [-0.2692028 , -0.7766612 ],
                                    ...,
                                    [-0.26909134, -0.7767676 ],
                                    [-0.26920405, -0.77668995],
                                    [-0.2691565 , -0.77663386]], dtype=float32))])

In [17]:
highest_predictions = np.argmax(predictions, axis=1)

AxisError: ignored

In [ ]:
from datasets import load_metric

metric = metric = load_metric("rouge")
metric.compute(predictions=highest_predictions, references=datasets["train"]["label"])